In [5]:
import numpy as np
import glob
import os
import uproot
import json

In [6]:
# Define auxilary functions 

def get_limits(fn): # Returns quantile vs limits
    f = uproot.open(fn)
    limit = f["limit"]['limit'].array(library="np")
    quant = f["limit"]['quantileExpected'].array(library="np")
    if limit.shape[0] == 1:
        return -1
    else:
        return np.stack([quant,limit]) 
    
def get_SUEP_file(Mass=125, mdark=2, t=1, decay='generic', path="../"): # Returns filename
    if t < 10:
        tem = "{0:.2f}".format(t)
    else:
        tem = "{0:.1f}".format(t)
    tem = str(tem).replace(".","p")
    fname = os.path.join(
        "{}higgsCombineGluGluToSUEP_HT1000_T{}_mS{:.3f}_mPhi{:.3f}_T{:.3f}_mode{}_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root".format(path, tem, Mass, mdark, t, decay)
    )
    if os.path.isfile(fname):
        return fname
    else:
        pass
     
def xs_scale(proc, era):
    xsec = 1.0
    with open(f"../config/xsections_{era}.json") as file:
            MC_xsecs = json.load(file)
    xsec  = MC_xsecs[proc]["xsec"]
    #xsec *= MC_xsecs[proc]["kr"]
    xsec *= MC_xsecs[proc]["br"]
    #xsec *= 1000.0
    assert xsec > 0, "{} has a null cross section!".format(proc)
    return xsec

# Get all SUEP masses for a given mdark and T
def get_params_from_dir(path="../", era='2018'):
    """
    Grabs the parameters [T, mS, mPhi, mode, xsec] from each file from combine
    from a specific directory. The 'era' parameter is just used for the
    cross sections, which for SUEP don't change between years.
    """

    # Get files
    combine_files = glob.glob("{}higgs*.root".format(path))

    # Filter out the parameters
    all_params = []
    for file in combine_files:
        file_trim = file.split("HT1000_")[1]
        proc = 'GluGluToSUEP_HT1000_' + file_trim.split('.Asymptotic')[0]
        xsec = xs_scale(proc, era)
        t = float(file_trim.split("_")[0].replace("p",".")[1:])
        Mass = float(file_trim.split("_")[1][2:])
        mphi = float(file_trim.split("_")[2][4:])
        mode = file_trim.split('_')[4][4:]
        all_params.append([t,Mass,mphi,mode, xsec])

    return all_params


def create_limit_table(params, path):
    """
    Expects params to be a list of lists of the shape: [t,Mass,mphi,mode, xsec].
    Returns table of limits.
    """

    rows = [['T', 'mS', 'mPhi', 'Mode', 'xsec', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma']]
    for param in params:
        lims = get_limits(get_SUEP_file(path=path, Mass=param[1], mdark=param[2], t=param[0], decay=param[3]))
        if lims.shape != (2,6):
            print(param, "has bad limits")
            continue

        lims = lims * param[-1]        

        _exp =lims[1][2]
        _s1p =lims[1][1] 
        _s1m =lims[1][3] 
        _s2p =lims[1][0] 
        _s2m =lims[1][4] 
        _obs =lims[1][5]   

        newline = [*param, _exp, _s2p, _s1p, _s1m, _s2m, _obs]
        rows.append(newline)
    
    return rows
    

In [7]:
# Take temperature, mphi/mdark, decay mode (in GeV) 
ofile = '../ClosureSyst/hep_data.txt'
itag = '../ClosureSyst/'

In [9]:
params = get_params_from_dir(itag)
table = create_limit_table(params, path=itag)
print(table[:10])

[['T', 'mS', 'mPhi', 'Mode', 'xsec', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma'], [3.0, 700.0, 3.0, 'generic', 0.621, 0.0009570465087890624, 0.00028038471937179564, 0.000497127948794514, 0.0018573460537008942, 0.0029061257326975466, 0.0009677029018846973], [3.0, 900.0, 3.0, 'generic', 0.269, 0.00041251373291015627, 0.00012085363268852234, 0.00021427600854076446, 0.0007972793565131724, 0.0012526084450073542, 0.0004135709929760125], [3.0, 125.0, 3.0, 'generic', 45.2, 3429.55, 1862.1384765625, 2462.163200378418, 4878.605334472657, 6710.590246582032, 2570.6049100668], [3.0, 400.0, 3.0, 'generic', 3.16, 0.05766845703125, 0.031086902618408206, 0.04167799070477486, 0.07927613504230976, 0.10498643219470978, 0.0695815610014679], [3.0, 200.0, 3.0, 'generic', 16.9, 118.03593749999999, 65.47305908203124, 86.41608095169066, 163.43882827758787, 220.13891687393186, 146.3154843375432], [3.0, 600.0, 3.0, 'generic', 1.0, 0.002105712890625, 0.000616908073425293, 0.0010937908664345741, 0.0040613897

In [10]:
with open(ofile, "w") as file:
    for row in table:
        # Convert the elements of the inner list to strings
        row_str = [str(element) for element in row]
        # Join the elements with a delimiter (e.g., comma) and write to the file
        file.write(",".join(row_str) + "\n")